<a href="https://colab.research.google.com/github/Sarak-Dahal/DataScienceAutomation/blob/main/Week_5_Assignment_Sarak_Dahal_3037512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DS Automation Assignment

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

*Optional* challenges:
- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's `input()` function, the `click` package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1.

In [19]:
# Importing required libraries
import pandas as pd

df = pd.read_csv('churn_data.csv',index_col='customerID')
df


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,
7590-VHVEG,1,No,Month-to-month,Electronic check,29.85,29.85,No
5575-GNVDE,34,Yes,One year,Mailed check,56.95,1889.50,No
3668-QPYBK,2,Yes,Month-to-month,Mailed check,53.85,108.15,Yes
7795-CFOCW,45,No,One year,Bank transfer (automatic),42.30,1840.75,No
9237-HQITU,2,Yes,Month-to-month,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...
6840-RESVB,24,Yes,One year,Mailed check,84.80,1990.50,No
2234-XADUH,72,Yes,One year,Credit card (automatic),103.20,7362.90,No
4801-JZAZL,11,No,Month-to-month,Electronic check,29.60,346.45,No


In [2]:
# Q1 Using Pycaret to find an ML algorithm that performs best on the data
!pip install pycaret==3.2
from pycaret.classification import *

In [20]:
# Setting up autoML
setup(data=df, target='Churn')

,Description,Value
0,Session id,7951
1,Target,Churn
2,Target type,Binary
3,Target mapping,"No: 0, Yes: 1"
4,Original data shape,"(7043, 7)"
5,Transformed data shape,"(7043, 12)"
6,Transformed train set shape,"(4930, 12)"
7,Transformed test set shape,"(2113, 12)"
8,Ordinal features,1
9,Numeric features,3


In [21]:
# Q2 Running autoML function to find best model
best_model = compare_models()


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7878,0.8314,0.7878,0.7779,0.7805,0.4220,0.4261,0.8430
ada,Ada Boost Classifier,0.7872,0.8309,0.7872,0.7761,0.7783,0.4139,0.4197,0.5460
lda,Linear Discriminant Analysis,0.7866,0.8210,0.7866,0.7767,0.7793,0.4190,0.4230,0.1450
gbc,Gradient Boosting Classifier,0.7850,0.8297,0.7850,0.7721,0.7744,0.4012,0.4082,0.5730
ridge,Ridge Classifier,0.7842,0.0000,0.7842,0.7695,0.7700,0.3856,0.3970,0.2450
lightgbm,Light Gradient Boosting Machine,0.7832,0.8188,0.7832,0.7729,0.7758,0.4099,0.4137,0.7180
rf,Random Forest Classifier,0.7694,0.7956,0.7694,0.7581,0.7613,0.3717,0.3755,0.6580
xgboost,Extreme Gradient Boosting,0.7680,0.8047,0.7680,0.7577,0.7610,0.3726,0.3756,0.2390
knn,K Neighbors Classifier,0.7639,0.7467,0.7639,0.7501,0.7537,0.3489,0.3539,0.1710
et,Extra Trees Classifier,0.7533,0.7739,0.7533,0.7451,0.7479,0.3420,0.3441,0.7000


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [22]:
best_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=7951, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
df.iloc[-2:-1]

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,
8361-LTMKD,4,Yes,Month-to-month,Mailed check,74.4,306.6,Yes


In [24]:
# Q3 Saving the model to disk
save_model(best_model, 'churn_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'MonthlyCharges',
                                              'TotalCharges'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,...
                                                               handle_missing='return_nan',
                                                               handle_unknown='value',
                                 

In [25]:
import pickle
with open('churn_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [26]:
with open('churn_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [27]:
loaded_lda = load_model('churn_model')

Transformation Pipeline and Model Successfully Loaded


In [28]:
new_data=df.iloc[-2:-1]

In [29]:
# Checking model with existing data
predict_model(loaded_lda, new_data)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,0,1.0000,1.0000,1.0000,nan,0.0000


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,prediction_label,prediction_score
customerID,,,,,,,,,
8361-LTMKD,4,Yes,Month-to-month,Mailed check,74.400002,306.600006,Yes,Yes,0.506


In [30]:
# Loading from predictor.py
from IPython.display import Code

Code('predictor.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):

    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    model = load_model('churn_model')
    predictions = predict_model(model, data=df)
    return predictions


if __name__ == "__main__":
    df = load_data('new_churn_data.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

In [31]:
%run predictor.py

Transformation Pipeline and Model Successfully Loaded


predictions:
            tenure  PhoneService  Contract  PaymentMethod  MonthlyCharges  \
customerID                                                                  
9305-CKSKC      22             1         0              2       97.400002   
1452-KNGVK       8             0         1              1       77.300003   
6723-OKKJM      28             1         0              0       28.250000   
7832-POPKP      62             1         0              2      101.699997   
6348-TACGU      10             0         0              1       51.150002   

            TotalCharges  charge_per_tenure prediction_label  prediction_score  
customerID                                                                      
9305-CKSKC    811.700012          36.895454              Yes            0.7923  
1452-KNGVK   1701.949951         212.743744              Yes            0.8826  
6723-OKKJM    250.899994           8.960714               No            0.7013  
7832-POPKP   3106.560059          50.10580

# Summary

Utilizing PyCaret, a popular autoML library, I performed model selection to identify the most effective machine learning algorithm for predicting churn within our dataset. Employing accuracy as the primary evaluation metric, I compared several classification algorithms to determine their performance.

Upon completion of model selection, the most promising candidate emerged as a Logistic Regression classifier, showcasing the highest accuracy score among the tested algorithms. Consequently, I saved this model to disk for future deployment and usage.

Following model selection and saving, I developed a Python script or module designed to facilitate churn prediction. This script contains a function capable of processing a Pandas DataFrame input and generating churn probability predictions for each record. Additionally, it prints out predictions for new data provided in a file named new_churn_data.csv.

Subsequently, I evaluated the script's performance by testing it with the new churn data and comparing its predictions against the true values provided as [1, 1, 0, 0, 1].
